In [1]:
import yfinance as yf

# Define the stock tickers
netflix_ticker = 'NFLX'
amazon_ticker = 'AMZN'

# Download data for Netflix and Amazon
netflix_data = yf.download(netflix_ticker, start='2024-04-01', end='2024-04-30')
amazon_data = yf.download(amazon_ticker, start='2024-04-01', end='2024-04-30')

# Extract relevant columns
netflix_prices = netflix_data[['Open', 'High', 'Low', 'Close', 'Volume']]
amazon_prices = amazon_data[['Open', 'High', 'Low', 'Close', 'Volume']]

# Print the first few rows of the data
print("Netflix stock prices:")
print(netflix_prices.head())

print("\nAmazon stock prices:")
print(amazon_prices.head())


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Netflix stock prices:
                  Open        High         Low       Close   Volume
Date                                                               
2024-04-01  608.000000  615.109985  605.570007  614.309998  2115900
2024-04-02  611.000000  615.030029  605.510010  614.210022  2029200
2024-04-03  612.750000  630.409973  611.500000  630.080017  2931200
2024-04-04  633.210022  638.000000  616.580017  617.140015  3064300
2024-04-05  624.919983  637.909973  622.710022  636.179993  3372800

Amazon stock prices:
                  Open        High         Low       Close    Volume
Date                                                                
2024-04-01  180.789993  183.000000  179.949997  180.970001  29174500
2024-04-02  179.070007  180.789993  178.380005  180.690002  32611500
2024-04-03  179.899994  182.869995  179.800003  182.410004  31046600
2024-04-04  184.000000  185.100006  180.000000  180.000000  41624300
2024-04-05  182.380005  186.270004  181.970001  185.070007  423352

In [2]:
amazon_prices.reset_index(inplace=True)

netflix_prices.reset_index(inplace=True)

In [3]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots


netflix_prices['Date'] = pd.to_datetime(netflix_prices['Date'])

# Create a subplot with 2 y-axes
fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])

# Add traces for Open, High, Low, Close
fig.add_trace(go.Scatter(x=netflix_prices['Date'], y=netflix_prices['Open'], name='Open', mode='lines+markers'), secondary_y=True)
fig.add_trace(go.Scatter(x=netflix_prices['Date'], y=netflix_prices['High'], name='High', mode='lines+markers'), secondary_y=True)
fig.add_trace(go.Scatter(x=netflix_prices['Date'], y=netflix_prices['Low'], name='Low', mode='lines+markers'), secondary_y=True)
fig.add_trace(go.Scatter(x=netflix_prices['Date'], y=netflix_prices['Close'], name='Close', mode='lines+markers'), secondary_y=True)

# Add trace for Volume using a bar chart
fig.add_trace(go.Bar(x=netflix_prices['Date'], y=netflix_prices['Volume'], name='Volume', marker_color='grey'), secondary_y=False)

# Update layout for a cleaner look
fig.update_layout(
    title='Netflix Stock Prices and Volume',
    xaxis_title='Date',
    yaxis_title='Price',
    yaxis2_title='Volume',
    legend_title='Data Type',
    template='plotly_white'
)

# Set y-axes titles
fig.update_yaxes(title_text="Price", secondary_y=True)
fig.update_yaxes(title_text="Volume", secondary_y=False)

fig.update_layout(height=800)
# Show the plot
fig.show()


In [5]:
import requests
import json

def get_latest_movies(api_key, company_name='Warner Bros.'):
    # Search for the production company by name to get the company ID
    search_url = f'https://api.themoviedb.org/3/search/company?api_key={api_key}&query={company_name}'
    search_response = requests.get(search_url)
    
    if search_response.status_code == 200:
        search_data = search_response.json()
        if search_data['results']:
            company_id = search_data['results'][0]['id']
        else:
            print(f"No company found with name {company_name}")
            return
    else:
        print(f"Error: {search_response.status_code}")
        return

    # Use the company ID to get the list of movies produced by the company
    movies_url = f'https://api.themoviedb.org/3/discover/movie?api_key={api_key}&with_companies={company_id}&sort_by=release_date.desc'
    movies_response = requests.get(movies_url)
    
    if movies_response.status_code == 200:
        movies_data = movies_response.json()
        return movies_data['results']
    else:
        print(f"Error: {movies_response.status_code}")
        return

# Replace 'YOUR_API_KEY' with your actual TMDB API key
api_key = '1b37fdfd40bfc02f3e734a5a77f5599d'

latest_movies = get_latest_movies(api_key)




In [13]:
from datetime import datetime
import csv
def filter_movies_by_release_date(movies):
    current_date = datetime.now().date()
    filtered_movies = [movie for movie in movies if datetime.strptime(movie['release_date'], '%Y-%m-%d').date() < current_date]
    return filtered_movies

def save_movies_to_csv(movies, filename='movies.csv'):
    keys = ['adult',
                'backdrop_path',
                'genre_ids',
                'id',
                'original_language',
                'original_title',
                'overview',
                'popularity',
                'poster_path',
                'release_date',
                'title',
                'video',
                'vote_average',
                'vote_count']
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        for movie in movies:
            filtered_movie = {key: movie.get(key, "") for key in keys}
            dict_writer.writerow(filtered_movie)
            
            
filtered_movies = filter_movies_by_release_date(latest_movies)
save_movies_to_csv(filtered_movies)